In [1]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.6/419.6 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp "/content/drive/MyDrive/NIR/train.csv" /content/train.csv
!cp "/content/drive/MyDrive/NIR/eval.csv" /content/eval.csv

In [4]:
!cp "/content/drive/MyDrive/NIR/bald-20230414T082728Z-001.zip" "/content/bald-20230414T082728Z-001.zip"

In [5]:
!unzip "/content/bald-20230414T082728Z-001.zip"

Archive:  /content/bald-20230414T082728Z-001.zip
  inflating: bald/optimizer.pt       
  inflating: bald/config.json        
  inflating: bald/rng_state.pth      
  inflating: bald/training_args.bin  
  inflating: bald/trainer_state.json  
  inflating: bald/scheduler.pt       
  inflating: bald/pytorch_model.bin  


In [6]:
import pandas as pd

train = pd.read_csv("/content/train.csv", encoding='utf-8')
eval = pd.read_csv("/content/eval.csv", encoding='utf-8')

In [7]:
import os
import random
from copy import deepcopy

import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [8]:
data = load_dataset("csv", data_files={'train': '/content/train.csv'})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7798e55faf20ea00/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

model = AutoModelForSequenceClassification.from_pretrained("/content/bald")

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=2,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = BaalTransformersTrainer(
    model=model,
    args=training_args,
    tokenizer=None
)

In [10]:
def tokenize_function(examples):
    return tokenizer(examples['last_part_content'], examples['part_body'], max_length=512, truncation=True, padding="max_length")

In [11]:
tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/14590 [00:00<?, ? examples/s]

In [12]:
output = trainer.predict(tokenized_data['train'].select(range(500, 14_590)))

In [13]:
predict_labels = np.argmax(output.predictions, axis=1)

In [14]:
# разметка 500 первых новостей train
for row_number, index in enumerate(train.index.tolist()[500:]):
  train.at[index, 'result_match'] = predict_labels[row_number]

In [18]:
data = pd.concat([train, eval]).sort_values(['match_id'], ascending = True)
data

,match_id,id,team1,team2,broadcast_name,first_part_content,last_part_content,part_body,after_match,relevance,result_match
500,18925,7274151,Локомотив,Химки,Россия. Премьер-лига,"Добрый день! Sports.ru поздравляет всех, кто п...",Драган Блатняк запустил мяч на трибуну со штра...,Дюрица получил травму в игре с «Химками». Защи...,1,1,0
501,18925,7274964,Локомотив,Химки,Россия. Премьер-лига,"Добрый день! Sports.ru поздравляет всех, кто п...",Драган Блатняк запустил мяч на трибуну со штра...,Антипенко получил травму в матче с «Локомотиво...,1,1,0
502,18935,7014478,Зенит,Штутгарт,перенесенные матчи,"Матчем со ""Штутгартом"" ""Зенит"" открывает европ...",Шибер впереди пока никак не зацепится за мяч. ...,Хусти забил в первом официальном матче в соста...,1,1,0
503,18956,7105473,Штутгарт,Зенит,перенесенные матчи,"Ответный матч 1/16 финала Кубка УЕФА ""Зенита"" ...",Гомес толкнул Крижанаца в верховой борьбе. Нап...,«Зенит» провел последнюю предматчевую трениров...,0,1,0
504,18956,7114149,Штутгарт,Зенит,перенесенные матчи,"Ответный матч 1/16 финала Кубка УЕФА ""Зенита"" ...",Гомес толкнул Крижанаца в верховой борьбе. Нап...,Семшов забил первый гол в составе «Зенита». По...,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
499,1104865,1040572409,Питтсбург,Сан-Хосе,Кубок Стэнли,"Привет всем, кто не представляет своей жизни б...","""Сан Хосе"" не помогает и замена вратаря. Плюс ...",НХЛ. Форвард «Сан-Хосе» Марло забросил 65-ю ша...,1,1,0
14586,1104866,1040621229,Питтсбург,Сан-Хосе,Кубок Стэнли,"Привет всем, кто не представляет своей жизни б...","Совершенно фантастический вклад в победы ""Питт...",НХЛ. Кубок Стэнли. Финал. «Питтсбург» вновь по...,0,1,1
14587,1104866,1040636713,Питтсбург,Сан-Хосе,Кубок Стэнли,"Привет всем, кто не представляет своей жизни б...","Совершенно фантастический вклад в победы ""Питт...",НХЛ. Форвард «Питтсбурга» Шери забил первый по...,1,1,1
14588,1104866,1040636981,Питтсбург,Сан-Хосе,Кубок Стэнли,"Привет всем, кто не представляет своей жизни б...","Совершенно фантастический вклад в победы ""Питт...",НХЛ. Мюррей одержал 13-ю победу в дебютном пле...,1,1,1


In [19]:
data['result_match'].value_counts()

1    10064
0     5026
Name: result_match, dtype: int64

In [20]:
data.to_csv('predicted_data.csv', index=False, encoding='utf-8')

In [21]:
!zip -r "/content/predicted_data.zip" "/content/predicted_data.csv"
!cp "/content/predicted_data.zip" "/content/drive/MyDrive/NIR/predicted_data.zip"

  adding: content/predicted_data.csv (deflated 86%)
